# О задании

## Тема
Объединение таблиц и сложные выборки

## Цель домашнего задания
- научиться работать с интервальными таблицы, уметь их джойнить с другими таблицами;
- научиться обрабатывать транзакции относительно какого-то промежутка времени.
 

## Условия
Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.

**За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**

Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

<font size=4>[**Шаг 1**](#Шаг-1)</font>

(Балл - 2) Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу *transaction_bd* со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции

(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции

(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента

(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала

(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу *customer_bd* со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента

(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте

(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте

(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента

(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

**Примечание для следующих шагов:** сумма транзакций не может быть null-ом, это всегда число.

<font size=4>[**Шаг 2**](#Шаг-2)</font>

**(Балл - 1 за каждый пункт)** Написать скрипты:

- a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
- d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
- e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

<font size=4>[**Шаг 3**](#Шаг-3)</font>

**(Балл - 3)** Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
 

## Сроки выполнения задания
До 09.04.2023, 23:59.

Импортируем библиотеки

In [1]:
import pandas as pd
import sqlite3

Создадим функции для подключения к базе данных, выполнения запросов и создании таблиц.

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [4]:
def sql_insert(con, entities, text_sql_insert_into_table):

    cursorObj = con.cursor()
    cursorObj.execute(text_sql_insert_into_table, entities)

## Шаг 1

Подключимся к базе данных

In [5]:
connection = create_connection('database.db')

Connection to SQLite DB successful


In [6]:
cursor = sqlite3.Cursor(connection)

Создадим таблицу transaction_bd

In [7]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS transaction_bd (
                                            "TRANSACTION_ID" INTEGER,
                                            "TX_DATETIME" NUMERIC,
                                            "CUSTOMER_ID" INTEGER,
                                            "TERMINAL_ID" INTEGER,
                                            "TX_AMOUNT" REAL NOT NULL
                                                        )
                     '''

In [8]:
execute_query(connection, sql_create_table)

Query executed successfully


Создадим таблицу customer_bd

In [9]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS customer_bd (
                                            "CLIENT_ID" INTEGER,
                                            "START_DT" NUMERIC,
                                            "END_DT" NUMERIC,
                                            "CLIENT_NAME" TEXT,
                                            "YEAR_BIRTH" TEXT
                                                        )
                     '''

In [10]:
execute_query(connection, sql_create_table)

Query executed successfully


Прочитаем данные из файлов и вставим их в таблицы

In [11]:
text_sql_insert_into_table = '''
                                 INSERT INTO transaction_bd(
                                                         TRANSACTION_ID,
                                                         TX_DATETIME,
                                                         CUSTOMER_ID,
                                                         TERMINAL_ID,
                                                         TX_AMOUNT
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [12]:
with open('transactions_for_dz2.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(','), text_sql_insert_into_table)
connection.commit()

In [13]:
text_sql_insert_into_table = '''
                                 INSERT INTO customer_bd(
                                                         START_DT,
                                                         END_DT,
                                                         CLIENT_NAME,
                                                         YEAR_BIRTH,
                                                         CLIENT_ID
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [14]:
with open('client_info.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(';'), text_sql_insert_into_table)
connection.commit()

## Шаг 2
Напишем скрипты:

a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [15]:
actual_date = '2023-05-01'

In [16]:
sql_task_a = f'''
            WITH
            
            actual_cusomers as
                (SELECT
                    CLIENT_ID
                FROM
                    customer_bd
                WHERE END_DT >= {actual_date}),
                
            sum_total as
                (SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE
                    CUSTOMER_ID in actual_cusomers
                GROUP by CUSTOMER_ID),

                max_total as
                (SELECT
                    *,
                    max(TOTAL_AMOUNT) over () as MAX_TOTAL_AMOUNT

                FROM
                    sum_total
                )


            SELECT
                CLIENT_NAME,
                MAX_TOTAL_AMOUNT,
                END_DT
            FROM
                max_total as m
            JOIN customer_bd as c on c.CLIENT_ID = m.CUSTOMER_ID
            WHERE TOTAL_AMOUNT = MAX_TOTAL_AMOUNT


            '''
data = pd.read_sql(sql_task_a, connection)
data

,CLIENT_NAME,MAX_TOTAL_AMOUNT,END_DT
0,Kylee,786115.87,2999-12-31


b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [17]:
sql_task_b = f'''
            WITH 
                      
            actual_cusomers as
                (SELECT
                    CLIENT_ID
                FROM
                    customer_bd
                WHERE END_DT >= {actual_date}),
                
            sum_total as
                (SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE
                    CUSTOMER_ID in actual_cusomers
                GROUP by CUSTOMER_ID),

                min_total as
                (SELECT
                    *,
                    min(TOTAL_AMOUNT) over () as MIN_TOTAL_AMOUNT

                FROM
                    sum_total
                )


            SELECT
                CLIENT_NAME,
                MIN_TOTAL_AMOUNT,
                END_DT
            FROM
                min_total as m
            JOIN customer_bd as c on c.CLIENT_ID = m.CUSTOMER_ID
            WHERE TOTAL_AMOUNT = MIN_TOTAL_AMOUNT


            '''
data = pd.read_sql(sql_task_b, connection)
data

,CLIENT_NAME,MIN_TOTAL_AMOUNT,END_DT
0,Kloe,30.48,2999-12-31


c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.

In [18]:
sql_task_c = f'''
                WITH 

                actual_cusomer as
                    (SELECT
                        CLIENT_ID
                    FROM
                        customer_bd
                    WHERE END_DT >= '2023-04-01')

                SELECT
                    CUSTOMER_ID,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd as t
                WHERE TX_DATETIME < '2023-04-01' AND CUSTOMER_ID in actual_cusomer AND substr(CUSTOMER_ID, 1, 3) = "111"
                GROUP by CUSTOMER_ID
                '''
data = pd.read_sql(sql_task_c, connection)
data

,CUSTOMER_ID,TOTAL_AMOUNT
0,1112,35273.16
1,1113,229558.85
2,1114,108622.52
3,1115,38690.76
4,1116,191203.59
5,1117,52751.58
6,1118,353003.39
7,1119,221192.71


d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.
e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [19]:
sql_task_d = f'''
                WITH 
                    actual_cusomer as
                        (SELECT
                            CLIENT_ID
                        FROM
                            customer_bd
                        WHERE END_DT >= '2023-05-01')


                SELECT
                    c.YEAR_BIRTH,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd as t
                JOIN customer_bd as c on t.CUSTOMER_ID = c.CLIENT_ID
                WHERE CUSTOMER_ID in actual_cusomer
                GROUP by c.YEAR_BIRTH
                ORDER by c.YEAR_BIRTH DESC
                '''

data = pd.read_sql(sql_task_d, connection)
data

,YEAR_BIRTH,TOTAL_AMOUNT
0,2000,8.632720e+07
1,1999,1.041507e+06
2,1998,6.016750e+05
3,1997,8.479943e+06
4,1996,1.483253e+06
5,1995,6.321200e+06
6,1994,6.244281e+06
7,1993,5.732058e+06
8,1992,5.251433e+06
9,1991,7.141074e+06


- e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [20]:
sql_task_e = f'''
                WITH 
                    actual_cusomer as
                        (SELECT
                            CLIENT_ID
                        FROM
                            customer_bd
                        WHERE END_DT >= '2023-05-01')

                SELECT
                    CUSTOMER_ID,
                    count(TRANSACTION_ID) as COUNT_TRANSACTION
                FROM
                    transaction_bd
                WHERE CUSTOMER_ID in actual_cusomer
                GROUP by CUSTOMER_ID
                ORDER by COUNT_TRANSACTION DESC
'''
data = pd.read_sql(sql_task_e, connection)
data

,CUSTOMER_ID,COUNT_TRANSACTION
0,2891,477
1,2580,464
2,1953,455
3,2932,454
4,3864,453
...,...,...
3731,1976,1
3732,1942,1
3733,1880,1
3734,1095,1


## Шаг 3

Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [21]:
sql_task_3 = f'''
                WITH 
                    actual_cusomer as
                        (SELECT
                            CLIENT_ID
                        FROM
                            customer_bd
                        WHERE END_DT >= '2023-05-01')

                SELECT
                    strftime('%m',TX_DATETIME) as MONTH,
                    sum(TX_AMOUNT) as TOTAL_AMOUNT
                FROM
                    transaction_bd
                WHERE CUSTOMER_ID in actual_cusomer
                GROUP by MONTH
                '''
data = pd.read_sql(sql_task_3, connection)
data

,MONTH,TOTAL_AMOUNT
0,01,1.159199e+08
1,02,1.062295e+08
2,03,1.150534e+08
3,04,7.261812e+07


In [22]:
connection.close()